<a href="https://colab.research.google.com/github/aaronjyang/transformers-testing/blob/main/LLaVa-loftus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load model and necessary imports

In [ ]:
# Pip installs
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-taojcrac
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-taojcrac
  Resolved https://github.com/huggingface/transformers.git to commit f11f57c92579aa311dbde5267bc0d8d6f2545f7b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10545344 sha256=aed141b8e70715f191e1a7556e19a17bb38fc7ce06160eaf7c006858e73a8223
  Stored in directory: /tmp/pip-ephem-wheel-cache-4vbkafdl/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found e

In [ ]:
# Imports
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
from PIL import Image
import numpy as np
import av
import numpy as np
from huggingface_hub import hf_hub_download
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [ ]:
# Load Model

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Load videos


In [ ]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [ ]:
videosIDs = ["000010", "000016", "000049", "000055", "000061", "000073", "000074",
          "000090", "000161", "000164", "000166", "000171", "000435", "000500",
          "000539", "000618"]
videos = ["/content/usable_videos/" + id + ".mp4" for id in videosIDs]
containers = [av.open(video) for video in videos]
indices = np.arange(0, 51)
clips = np.asarray([read_video_pyav(containers[i], indices) for i in range(len(containers))])
np.shape(clips)

(16, 50, 720, 1280, 3)

# Test videos (optional)


In [ ]:
# np array with shape (frames, height, width, channels)
video = clips[2]

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=100)
HTML(anim.to_html5_video())

# Misc. Functions

In [ ]:
def generate_response(conversation, video, max_new_tokens = 100, do_sample= True, top_p = 0.9):
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor([prompt], videos=video, padding=True, return_tensors="pt").to(model.device)
  generate_kwargs = {"max_new_tokens": max_new_tokens, "do_sample": do_sample, "top_p": top_p}
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)
  return generated_text[0][len(prompt) - 7 + 1:]

# Experiment

In [ ]:
# Questions
questions = [ "Give an account of the accident you have just seen in the provided video.",
              "Does the video happen at day or at night?",
              "What is the weather in the video?",
              "Was ego vehicle itself involved in the car accident?",
              "Were any pedestrians involved in the accident?",
              "About how fast were the cars going when they smashed into each other?"]

In [ ]:
# Create the template for the conversation
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Does a car accident occur in the provided video?"},
              {"type": "video"},
              ],
      },
]

In [ ]:
# Generate model-readable prompt
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor([prompt], videos=[clips[2]], padding=True, return_tensors="pt").to(model.device)

In [ ]:
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}
output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=True)

In [ ]:
print(generated_text)

['USER: \nDoes a car accident occur in the provided video? ASSISTANT: Yes, a car accident is shown in the provided video, where a vehicle appears to collide with another vehicle, causing the second car to veer off the road and into a skid. The accident is clearly visible on the road and the impact of the collision is significant enough to cause damage to both cars involved.']


In [ ]:
cut_output = generated_text[0][len(prompt) - 7 + 1:]

In [ ]:
outputs = {"Does a car accident occur in the provided video?" : cut_output}

In [ ]:
assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": cut_output},
      ],
  }
conversation.append(assistant_response)

In [ ]:
for question in questions:
  next_question = {
      "role": "user",
          "content": [
              {"type": "text", "text": question},
          ],
  }
  conversation.append(next_question)
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor([prompt], videos=[clips[2]], padding=True, return_tensors="pt").to(model.device)
  generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)

  cut_output = generated_text[0][len(prompt) - 7 + 1:]
  outputs[question] = cut_output

  assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": cut_output},
      ],
  }
  conversation.append(assistant_response)


In [ ]:
outputs

{'Does a car accident occur in the provided video?': 'Yes, a car accident is shown in the provided video, where a vehicle appears to collide with another vehicle, causing the second car to veer off the road and into a skid. The accident is clearly visible on the road and the impact of the collision is significant enough to cause damage to both cars involved.',
 'Give an account of the accident you have just seen in the provided video.': 'In the video, a car accident occurs where a vehicle swerves off the road and collides with another car, resulting in significant damage to both vehicles. The accident takes place at night, and it is visible in the provided video, showing the vehicles collide and the aftermath of the impact, including the skid marks left behind on the road. The image is a still frame from the video, not a separate or different depiction of the event.',
 'Does the video happen at day or at night?': 'The video is set at night, as evidenced by the darkness visible in the f